### DataDive September 2021
### Housing Insecurity project

Notebook for Data Exploration and Sample Maps

Data:  Miami-Dade county, census-tract level housing data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
import seaborn as sns
import geopandas as gpd
import folium
from folium import plugins

In [4]:
data_dir = './data/housing_insecurity/Housing_Loss_Data_202109/'
print(os.listdir(data_dir))

['data_dictionary.xlsx', 'hillsborough_fl_2010_tracts_formatted.geojson', 'hillsborough_fl_processed_2017_to_2019_20210916.csv', 'miami_dade_fl_2010_tracts_formatted.geojson', 'miami_dade_fl_processed_2017_to_2019_20210916.csv', 'orange_fl_2010_tracts_formatted.geojson', 'orange_fl_processed_2017_to_2019_20210916.csv']


In [5]:
output_dir = '../figures/'

Load data

In [126]:
geo_file = data_dir + 'miami_dade_fl_2010_tracts_formatted.geojson'
geo_data = gpd.read_file(geo_file)
geo_data.set_index('census_tract_GEOID', inplace=True)

In [127]:
geo_data.head()

,geometry
census_tract_GEOID,
12086008306,"POLYGON ((-80.36583 25.62142, -80.36587 25.620..."
12086008601,"POLYGON ((-80.35977 25.73252, -80.35974 25.732..."
12086008602,"POLYGON ((-80.38287 25.71492, -80.38279 25.712..."
12086010005,"POLYGON ((-80.29386 25.93929, -80.29385 25.939..."
12086010009,"POLYGON ((-80.27845 25.96218, -80.27842 25.961..."


In [148]:
data_file = data_dir + 'miami_dade_fl_processed_2017_to_2019_20210916.csv'
housing_data = pd.read_csv(data_file)
housing_data.shape

(519, 73)

In [149]:
housing_data['census_tract_GEOID'] = housing_data['census_tract_GEOID'].astype(str)

In [190]:
# Replace missing value codes with NaN

var_list = ['median-gross-rent','median-household-income','median-property-value','median-monthly-housing-cost']

for var in var_list:
    housing_data[var][housing_data[var] < -999999] = np.nan

C:\Users\jayka\.conda\envs\davm\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jayka\.conda\envs\davm\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jayka\.conda\envs\davm\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jayka\.conda\envs\davm\lib\site-packages\ipykernel_launcher.py:6

In [191]:
housing_data.head()

,census_tract_GEOID,total-households,total-renter-occupied-households,total-owner-occupied-households,total-owner-occupied-households-mortgage,median-gross-rent,median-household-income,median-property-value,median-monthly-housing-cost,pct-white,...,lien-foreclosure-rate-2019,avg-eviction-rate,ratio-to-mean-foreclosure-rate,ratio-to-mean-eviction-rate,avg-housing-loss-rate,evictions-pct-total-housing-loss,housing-loss-index,county_GEOID,county,state
0,12086000211,931,369,562,352,1592.0,53533.0,240400.0,1728.0,710,...,0.355872,2.529359,0.650786,1.733086,2.034212,0.636364,1.130385,12086,Miami-Dade County,Florida
1,12086000212,1668,1373,295,201,1109.0,33958.0,179900.0,1389.0,887,...,1.016949,3.641661,1.068455,2.495222,3.494282,0.909091,1.941727,12086,Miami-Dade County,Florida
2,12086000213,1379,694,685,356,1291.0,40250.0,254900.0,1701.0,696,...,0.437956,1.585014,1.367382,1.086033,2.126984,0.492537,1.181937,12086,Miami-Dade County,Florida
3,12086000214,2238,1209,1029,649,1135.0,39962.0,147800.0,1299.0,1362,...,0.194363,1.681831,0.750059,1.152370,1.704342,0.642105,0.947080,12086,Miami-Dade County,Florida
4,12086000128,1364,726,638,396,1349.0,63889.0,205900.0,1610.0,1240,...,0.156740,0.459137,0.831560,0.314595,0.980392,0.303030,0.544791,12086,Miami-Dade County,Florida


In [192]:
housing_data.describe()

,total-households,total-renter-occupied-households,total-owner-occupied-households,total-owner-occupied-households-mortgage,median-gross-rent,median-household-income,median-property-value,median-monthly-housing-cost,pct-white,pct-af-am,...,lien-foreclosure-rate-2017,lien-foreclosure-rate-2018,lien-foreclosure-rate-2019,avg-eviction-rate,ratio-to-mean-foreclosure-rate,ratio-to-mean-eviction-rate,avg-housing-loss-rate,evictions-pct-total-housing-loss,housing-loss-index,county_GEOID
count,519.000000,519.000000,519.000000,519.000000,500.00000,506.000000,5.010000e+02,495.000000,519.000000,519.000000,...,410.000000,410.000000,410.000000,501.000000,501.000000,501.000000,492.000000,492.000000,492.000000,519.0
mean,1702.065511,831.290944,870.774566,534.870906,1501.01200,59890.992095,3.384770e+05,1951.137374,1309.630058,269.450867,...,0.355023,0.395378,0.369703,inf,inf,inf,1.882098,0.433712,1.045857,12086.0
std,743.936825,616.581542,517.058581,371.030872,479.82237,32597.422195,2.552797e+05,746.378358,738.074403,425.150663,...,0.808607,0.877375,0.814758,NaN,NaN,NaN,1.110809,0.245084,0.617262,0.0
min,0.000000,0.000000,0.000000,0.000000,245.00000,14848.000000,2.400000e+04,770.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.087873,0.066489,0.060210,0.166806,0.046154,0.092692,12086.0
25%,1257.500000,359.500000,464.000000,264.000000,1143.50000,36799.750000,2.017000e+05,1428.000000,855.000000,17.000000,...,0.000000,0.000000,0.000000,0.636787,0.679081,0.436319,1.120570,0.224306,0.622686,12086.0
50%,1617.000000,737.000000,847.000000,491.000000,1414.50000,52669.000000,2.735000e+05,1763.000000,1280.000000,69.000000,...,0.106731,0.139179,0.124031,1.154562,0.994675,0.791092,1.613997,0.382398,0.896877,12086.0
75%,2078.500000,1152.500000,1168.000000,724.500000,1742.75000,73578.250000,3.705000e+05,2220.500000,1742.000000,287.000000,...,0.303721,0.386102,0.355139,1.989390,1.449852,1.363106,2.290207,0.637799,1.272638,12086.0
max,5808.000000,4449.000000,3088.000000,2217.000000,3501.00000,250001.000000,2.000001e+06,4001.000000,5118.000000,2565.000000,...,6.756757,8.737864,6.164384,inf,inf,inf,6.909582,0.988281,3.839564,12086.0


In [193]:
merged_data = pd.merge(geo_data, housing_data, left_index=True, right_on='census_tract_GEOID')

Census Tracts with missing Housing Loss Index
(probably missing one or more required data items)

In [194]:
housing_data[np.isnan(housing_data['housing-loss-index'])]

,census_tract_GEOID,total-households,total-renter-occupied-households,total-owner-occupied-households,total-owner-occupied-households-mortgage,median-gross-rent,median-household-income,median-property-value,median-monthly-housing-cost,pct-white,...,lien-foreclosure-rate-2019,avg-eviction-rate,ratio-to-mean-foreclosure-rate,ratio-to-mean-eviction-rate,avg-housing-loss-rate,evictions-pct-total-housing-loss,housing-loss-index,county_GEOID,county,state
78,12086008904,0,0,0,0,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12086,Miami-Dade County,Florida
130,12086004901,1444,1361,83,60,1137.0,29590.0,86900.0,NaN,1359,...,NaN,0.636787,NaN,0.436319,NaN,NaN,NaN,12086,Miami-Dade County,Florida
136,12086000807,1561,1448,113,45,927.0,21757.0,273600.0,1338.0,1461,...,0.000000,0.713628,NaN,0.488969,NaN,NaN,NaN,12086,Miami-Dade County,Florida
146,12086014000,90,90,0,0,2041.0,97679.0,NaN,NaN,72,...,NaN,NaN,inf,NaN,NaN,NaN,NaN,12086,Miami-Dade County,Florida
148,12086014100,0,0,0,0,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12086,Miami-Dade County,Florida
233,12086010015,1124,474,650,116,882.0,32984.0,52100.0,1000.0,979,...,NaN,0.316456,NaN,0.216832,NaN,NaN,NaN,12086,Miami-Dade County,Florida
251,12086008415,1512,32,1480,1014,2250.0,145625.0,598100.0,3219.0,1468,...,0.000000,NaN,0.254153,NaN,NaN,NaN,NaN,12086,Miami-Dade County,Florida
267,12086008101,1160,64,1096,693,NaN,194815.0,972200.0,4001.0,1108,...,0.182482,NaN,0.516497,NaN,NaN,NaN,NaN,12086,Miami-Dade County,Florida
268,12086008102,1478,243,1235,819,1431.0,141711.0,788000.0,3368.0,1251,...,NaN,NaN,0.559406,NaN,NaN,NaN,NaN,12086,Miami-Dade County,Florida
269,12086008205,1174,89,1085,589,2046.0,170385.0,875400.0,3747.0,1042,...,0.092166,NaN,1.045235,NaN,NaN,NaN,NaN,12086,Miami-Dade County,Florida


In [211]:
housing_data[np.isnan(housing_data['housing-loss-index'])]['census_tract_GEOID']

78     12086008904
130    12086004901
136    12086000807
146    12086014000
148    12086014100
233    12086010015
251    12086008415
267    12086008101
268    12086008102
269    12086008205
271    12086008207
318    12086980300
320    12086980100
321    12086980200
322    12086980400
323    12086980500
324    12086980600
325    12086980700
326    12086980800
327    12086980900
352    12086981000
353    12086990000
408    12086004608
422    12086005410
461    12086007801
462    12086007901
503    12086003706
Name: census_tract_GEOID, dtype: object

#### Maps

In [212]:
map_center = [25.7712, -80.1895]

In [213]:
m = folium.Map(location=map_center, tiles='openstreetmap',
                   zoom_start=9, min_zoom=9) 


In [214]:
# Map of Housing Loss Index

cpm = folium.Choropleth(
    geo_data=geo_data.to_json(),
    name='housing-loss-index',
    data=housing_data,
    columns=['census_tract_GEOID', 'housing-loss-index'],
    key_on='feature.id',
    fill_color='Reds',
    nan_fill_color='grey',
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name='Housing Loss Index'
)
m.add_child(cpm)

folium.LayerControl().add_to(m)
m

In [204]:
cpm2 = folium.Choropleth(
    geo_data=geo_data.to_json(),
    name='median-gross-rent',
    data=housing_data,
    columns=['census_tract_GEOID', 'median-gross-rent'],
    key_on='feature.id',
    fill_color='Greens',
    nan_fill_color='grey',
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name='Median Gross Rent'
)
#m.add_child(cpm2)

#folium.LayerControl().add_to(m)

#m